# Object Detection

The method used in this object detection algorithm is EfficientDet-Lite, a pretrained object detector. This is trained on COCO 2017 eval set and achieves mAP 41.5. 

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
from scipy.spatial import distance as dist
import tensorflow_hub as hub
import cv2
import glob
import numpy
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os

In [ ]:
detector = hub.load("https://tfhub.dev/tensorflow/efficientdet/lite4/detection/1") #link with image classifier
#link is subject to change due to updates.
#If this cell throws an error, increase the number in front of "lite" by 1. 
#Ex: lite4 -> lite5

In [ ]:
labels = pd.read_csv('/content/gdrive/MyDrive/labels.csv', sep=';', index_col='ID') # Download the label file from github
labels = labels['SUPER CATEGORY']

In [ ]:
def detect_objects(input_filepath, output_filepath, width=1028, height=1028, show=False):
  # load data and preprocessing
  img = cv2.imread(input_filepath)
  if width is None or height == None:
    inp = img
  else:
    inp = cv2.resize(img, (width, height))
  rgb = cv2.cvtColor(inp, cv2.COLOR_BGR2RGB)
  rgb_tensor = tf.convert_to_tensor(rgb, dtype = tf.uint8)
  rgb_tensor = tf.expand_dims(rgb_tensor, 0)

  # run model
  boxes, scores, classes, num_detection = detector(rgb_tensor)

  # post processing
  pred_labels = classes.numpy().astype('int')[0] 
  pred_labels = [labels[i] for i in pred_labels]
  pred_boxes = boxes.numpy()[0].astype('int')
  pred_scores = scores.numpy()[0]
  
  # detect_objects.checker = check_animal_person(pred_labels, pred_scores)
  
  # draw boxes on the figure
  output_image = rgb
  detect_objects.fileID = output_filepath.split('/')[-1]
  human_coordinates = []
  
  # animal_coordinates = []
  for score, (ymin, xmin, ymax, xmax), label in zip(pred_scores, pred_boxes, pred_labels):
    if score < 0.25:
      continue

    if score > 0.25 and label == 'person':
      human_coordinates.append((ymin, xmin, ymax, xmax))
      
    # if score > 0.25 and label == 'animal':
      # animal_coordinates.append((ymin, xmin, ymax, xmax))

    # draw box    
    # i am thinking just 
    # score_txt = f'{100 * round(score)}%'
    score_txt = f'{round(100 * score)}%'
    output_image = cv2.rectangle(output_image, (xmin, ymax),(xmax, ymin),(0,255,0),2)
  

    # put text
    font = cv2.FONT_HERSHEY_SIMPLEX
    output_image = cv2.putText(output_image, label, (xmin, ymax-10), font, 1.5, (255,0,0), 2, cv2.LINE_AA)
    # output_image = cv2.putText(output_image, score_txt, (xmax, ymax-10), font, 1.5, (255,0,0), 2, cv2.LINE_AA)
      
  # save figure to the specified path
  ### if the predicted labels do not contain human and animals at the same time 
  ### we will not save the image to the output file
  cv2_output_image = cv2.cvtColor(output_image, cv2.COLOR_RGB2BGR)
  cv2.imwrite(output_filepath, cv2_output_image)
  # print(human_coordinates)
  # print(animal_coordinates)
  if len(human_coordinates) >=2:
    detect_objects.human_target, detect_objects.animal_target, detect_objects.min_dis, detect_objects.human_width, detect_objects.human_length, detect_objects.animal_width, detect_objects.animal_length, detect_objects.human_area, detect_objects.animal_area, detect_objects.area_ratio = feature_creation(human_coordinates)
    print(detect_objects.fileID)
    print(detect_objects.min_dis)
    print(human_coordinates)
  
  # show figure if "show" is true
  if show:
    plt.figure(figsize=(10,10))
    plt.imshow(output_image)

In [ ]:
# You could create feature you want here. 
def feature_creation(human_coordinates):
  min_dis = 100000
  human_index = 0
  human2_index = 0
  for i in range(len(human_coordinates)-1):
    for j in range(i+1, len(human_coordinates)):
      human_mid_x = (human_coordinates[i][1] + human_coordinates[i][3])/2
      human_mid_y = (human_coordinates[i][0] + human_coordinates[i][2])/2
      human2_mid_x = (human_coordinates[j][1] + human_coordinates[j][3])/2
      human2_mid_y = (human_coordinates[j][0] + human_coordinates[j][2])/2
      distance = ((human_mid_x - human2_mid_x)**2 + (human_mid_y - human2_mid_y)**2)**0.5
      if distance < min_dis:
        min_dis = distance
        human_index = i
        human2_index = j
      else:
        continue
  print(human_index)
  print(human2_index)
  human_target = human_coordinates[human_index]
  animal_target = human_coordinates[human2_index]
  human_width = abs(human_target[3]-human_target[1])
  human_length = abs(human_target[2]-human_target[0])
  animal_width = abs(animal_target[3]-animal_target[1])
  animal_length = abs(animal_target[2]-animal_target[0])
  human_area = human_width * human_length
  animal_area = animal_width * animal_length

  area_list = [human_area, animal_area]
  area_ratio = max(area_list) / min(area_list)

  return human_target, animal_target, min_dis, human_width, human_length, animal_width, animal_length, human_area, animal_area, area_ratio

In [ ]:
# os.chdir('/content/gdrive/My Drive/Pig_new')
# !ls

In [ ]:
# !pip install natsort

In [ ]:
# from natsort import natsorted

In [ ]:
# os.listdir casues some randomness when reading the filenames
# This function helps make sure it will follow the order exactly in the drive 
# Then, the table can be directly used along with the images when training
# import re
# numbers = re.compile(r'(\d+)')
# def numericalSort(value):
#     parts = numbers.split(value)
#     parts[1::2] = map(int, parts[1::2])
#     return parts

In [ ]:
import re
def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(data, key=alphanum_key)

In [ ]:
input_folder = "/content/gdrive/MyDrive/human-human-raw/Non_violation" # Set to the location of input file
filenames = sorted_alphanumeric(os.listdir(input_folder))
extensions = ['jpg', 'bmp', 'png', 'JPG', 'JPEG', 'jpeg']
filenames = [filename for filename in filenames] #if image type is not .jpg, replace with appropirate photo type
# filenames = sorted(filenames, key=numericalSort)
# filenames = natsorted(filenames)
len(filenames)

14

In [ ]:
output_folder = "/content/gdrive/MyDrive/human-human-raw/Non_violation_detected" # Create a destination folder for images with objects detected
os.makedirs(output_folder, exist_ok=True)

In [ ]:
fileID = []
min_dis = []
human_width = []
human_length = []
human_area = []
animal_width = []
animal_length = []
animal_area = []
area_ratio = []
human_target = []
animal_target = []

for index, filename in enumerate(filenames, 1):
  print("Processing {} / {} - {}".format(index, len(filenames), filename))
  detect_objects(
    input_filepath = os.path.join(input_folder, filename),
    output_filepath = os.path.join(output_folder, filename),
    show=False, # you can turn this on if needed  
  )

  fileID.append(detect_objects.fileID)
  min_dis.append(detect_objects.min_dis)
  human_width.append(detect_objects.human_width)
  human_length.append(detect_objects.human_length)
  animal_width.append(detect_objects.animal_width)
  animal_length.append(detect_objects.animal_length)
  human_area.append(detect_objects.human_area)
  animal_area.append(detect_objects.animal_area)
  area_ratio.append(detect_objects.area_ratio)
  human_target.append(detect_objects.human_target)
  animal_target.append(detect_objects.animal_target)


Processing 1 / 14 - download (1).jpg
0
2
download (1).jpg
191.4451618610405
[(66, 15, 367, 187), (89, 851, 378, 1016), (58, 21, 756, 219)]
Processing 2 / 14 - download (2).jpg
0
1
download (2).jpg
436.0071673722807
[(499, 270, 866, 389), (512, 710, 858, 821)]
Processing 3 / 14 - images (1).jpg
0
1
images (1).jpg
471.0883675065645
[(0, 657, 362, 792), (0, 184, 806, 434)]
Processing 4 / 14 - images (2).jpg
1
2
images (2).jpg
39.05444917035702
[(295, 163, 646, 242), (287, 48, 545, 107), (304, 15, 503, 66), (296, 89, 581, 150), (225, 790, 701, 911), (255, 253, 713, 349)]
Processing 5 / 14 - images (4).jpg
0
1
images (4).jpg
540.5018501355939
[(15, 512, 1020, 979), (323, 113, 1024, 343)]
Processing 6 / 14 - images (5).jpg
0
1
images (5).jpg
599.1804819918619
[(69, 191, 659, 347), (55, 733, 1010, 955)]
Processing 7 / 14 - images (6).jpg
Processing 8 / 14 - images (11).jpg
1
2
images (11).jpg
457.5784085815239
[(472, 0, 648, 183), (340, 527, 444, 577), (492, 904, 686, 1026)]
Processing 9 / 14

In [ ]:
df = pd.DataFrame(list(zip(fileID, min_dis, human_target, human_width, human_length, human_area, animal_target, animal_width, animal_length, animal_area, area_ratio)),
               columns =['File Name', 'Closest Distance', 'Human Target', 'Human Width', 'Human Length', 
                         'Human Area', 'Animal Taget', 'Animal Width', 'Animal Length', 'Animal Area', 'Area Ratio'])
print(df.head())
print(df.shape)

          File Name  Closest Distance          Human Target  Human Width  \
0  download (1).jpg        191.445162    (66, 15, 367, 187)          172   
1  download (2).jpg        436.007167  (499, 270, 866, 389)          119   
2    images (1).jpg        471.088368    (0, 657, 362, 792)          135   
3    images (2).jpg         39.054449   (287, 48, 545, 107)           59   
4    images (4).jpg        540.501850  (15, 512, 1020, 979)          467   

   Human Length  Human Area           Animal Taget  Animal Width  \
0           301       51772     (58, 21, 756, 219)           198   
1           367       43673   (512, 710, 858, 821)           111   
2           362       48870     (0, 184, 806, 434)           250   
3           258       15222     (304, 15, 503, 66)            51   
4          1005      469335  (323, 113, 1024, 343)           230   

   Animal Length  Animal Area  Area Ratio  
0            698       138204    2.669474  
1            346        38406    1.137140  
2 

In [ ]:
label = [0] * df.shape[0]
df['Label'] = label
df

,File Name,Closest Distance,Human Target,Human Width,Human Length,Human Area,Animal Taget,Animal Width,Animal Length,Animal Area,Area Ratio,Label
0,download (1).jpg,191.445162,"(66, 15, 367, 187)",172,301,51772,"(58, 21, 756, 219)",198,698,138204,2.669474,0
1,download (2).jpg,436.007167,"(499, 270, 866, 389)",119,367,43673,"(512, 710, 858, 821)",111,346,38406,1.137140,0
2,images (1).jpg,471.088368,"(0, 657, 362, 792)",135,362,48870,"(0, 184, 806, 434)",250,806,201500,4.123184,0
3,images (2).jpg,39.054449,"(287, 48, 545, 107)",59,258,15222,"(304, 15, 503, 66)",51,199,10149,1.499852,0
4,images (4).jpg,540.501850,"(15, 512, 1020, 979)",467,1005,469335,"(323, 113, 1024, 343)",230,701,161230,2.910966,0
5,images (5).jpg,599.180482,"(69, 191, 659, 347)",156,590,92040,"(55, 733, 1010, 955)",222,955,212010,2.303455,0
6,images (6).jpg,599.180482,"(69, 191, 659, 347)",156,590,92040,"(55, 733, 1010, 955)",222,955,212010,2.303455,0
7,images (11).jpg,457.578409,"(340, 527, 444, 577)",50,104,5200,"(492, 904, 686, 1026)",122,194,23668,4.551538,0
8,images (12).jpg,632.401178,"(66, 404, 350, 680)",276,284,78384,"(651, 172, 1026, 814)",642,375,240750,3.071418,0
9,images (19).jpg,575.460251,"(199, 312, 960, 455)",143,761,108823,"(38, 865, 828, 1015)",150,790,118500,1.088924,0


In [ ]:
df.to_csv('Non-Violation.csv')
!cp Non-Violation.csv "/content/gdrive/MyDrive/human-human-raw/"